In [2]:
%load_ext autoreload
%autoreload 2
from EPECinterface import *
from gurobipy import *
import itertools
import csv

## Sets

In [3]:
dirty = ["coal", "gas"]
green = ["wind", "solar"]

countries = ["c1", "c2"]
producers = ["p1_1", "p1_2", "p2_1", "p2_2"]

domesticity = tuplelist([("c1","p1_1"), ("c1","p1_2"),
               ("c2","p2_1"), ("c2","p2_2")
              ])

scenario = ["s"+str(i+1) for i in range(2
                                       )]

data = inputData(dirty, green, countries, producers, domesticity, scenario)

## Loading Example data

In [4]:
dataBase = [15 ,25 ,15 ,25 ,15 ,25 ,15 ,25 ,1 ,4 ,1 ,4 ,1 ,4 ,1 ,4 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,100 ,100 ,100 ,100 ,3 ,2 ,0 ,0 ,800 ,1200 ,1200 ,800 ,1 ,1 ,1 ,1 ,200 ,200 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]
data.buildFromList(dataBase)
data

INSTANCE OF EPECinterface.inputData
***********************************

dirty ---------- ['coal', 'gas']

green ---------- ['wind', 'solar']

energy ---------- ['coal', 'gas', 'wind', 'solar']

countries ---------- ['c1', 'c2']

producers ---------- ['p1_1', 'p1_2', 'p2_1', 'p2_2']

domesticity ---------- <gurobi.tuplelist (4 tuples, 2 values each):
 ( c1 , p1_1 )
 ( c1 , p1_2 )
 ( c2 , p2_1 )
 ( c2 , p2_2 )
>

scenario ---------- ['s1', 's2']

LinInvCost ---------- {('p1_1', 'wind'): 15, ('p1_1', 'solar'): 25, ('p1_2', 'wind'): 15, ('p1_2', 'solar'): 25, ('p2_1', 'wind'): 15, ('p2_1', 'solar'): 25, ('p2_2', 'wind'): 15, ('p2_2', 'solar'): 25}

QuadInvCost ---------- {('p1_1', 'wind'): 1, ('p1_1', 'solar'): 4, ('p1_2', 'wind'): 1, ('p1_2', 'solar'): 4, ('p2_1', 'wind'): 1, ('p2_1', 'solar'): 4, ('p2_2', 'wind'): 1, ('p2_2', 'solar'): 4}

LinProdCost ---------- {('p1_1', 'coal'): 4, ('p1_1', 'gas'): 6, ('p1_1', 'wind'): 0, ('p1_1', 'solar'): 0, ('p1_2', 'coal'): 4, ('p1_2', 'gas'): 6, 

In [4]:
mylist = readInputFromFile("inputs.csv")
len(mylist)

15

In [5]:
baseData = mylist[0]
data = inputData(dirty, green, countries, producers, domesticity, scenario)
data.buildFromList(baseData)

In [58]:
M = Model("infeasModel")
x = M.addVar(name="x")
y = M.addVar(name="x")
z = M.addVar(name="x")
M.addConstr(x+y == 3)
M.addConstr(y+z == 3)
M.addConstr(x+2*y+z <= 5)
M.update()
M

<gurobi.Model Continuous instance infeasModel: 3 constrs, 3 vars, No parameter changes>

In [68]:
"_".join( [str(s) for s in (1,5,6,-1)] )

'1_5_6_-1'

In [259]:
solution = solveMPECs(mylist)

In [ ]:
with open("output.txt","w") as file:
    for vals in solution:
        file.write(",".join([str(vv) for vv in vals]))
        file.write('\n')

In [3]:
M = Model()


Using license file /home/sriram/gurobi.lic
Academic license - for non-commercial use only


In [7]:
a = [1,2,3]
b = a.copy()
b[1]=24
print(a,b)

[1, 2, 3] [1, 24, 3]


# Production zone

In [46]:
dataBase = [15 ,25 ,15 ,25 ,15 ,25 ,15 ,25 ,1 ,4 ,1 ,4 ,1 ,4 ,1 ,4 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,4 ,6 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.2 ,0.2 ,0 ,0 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,0.4 ,0.6 ,0.6 ,0.4 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,200 ,200 ,20 ,20 ,100 ,100 ,100 ,100 ,3 ,2 ,0 ,0 ,800 ,1200 ,1200 ,800 ,1 ,1 ,1 ,1 ,200 ,200 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]

def invSubsidy(data,scen):
    """
    Subsidy scenario
    
    PARAMETERS
    data: 
        inputData objecti
    scen:
        -1 or 0 or 1 implying low, medium or high subsidy
    """
    fact = (scen + 1)*1.0
    for xx in data.LinInvCost:
        data.LinInvCost[xx] = data.LinInvCost[xx]*(1 - fact*5/100)
        data.QuadInvCost[xx] = data.LinInvCost[xx]*(1 - fact*5/100)
    return data

def demSlope(data, scen):
    """
    Demand Slope altering scenario
    
    PARAMETERS
    data: 
        inputData objecti
    scen:
        -1 or 0 or 1 implying low, medium or high demand Slope
    """
    fact = 0.1*scen
    for xx in data.DemSlope:
        data.DemSlope[xx] = data.DemSlope[xx]*(1 + fact)
        data.QuadInvCost[xx] = data.DemSlope[xx]*(1 + fact)
    return data

def stdDevnCapFac(data, scen):
    """
    Standard deviation of the capacity factor in scenarios
    
    PARAMETERS
    data: 
        inputData objecti
    scen:
        -1 or 0 or 1 implying low, medium or high capacity factors
    
    WARNING
        Only works for 2 scenario programs
    """
    fact = 0.1*(2+scen)
    for cc in data.producers:
        data.DemSlope[pp, 'wind','s1'] = 0.5 - fact
        data.DemSlope[pp, 'slope','s1'] = 0.5 + fact
        data.DemSlope[pp, 'wind','s2'] = 0.5 + fact
        data.DemSlope[pp, 'slope','s2'] = 0.5 - fact
    return data

def emmInv(data, scen, cc):
    fact = scen + 1
    data.EmissionValue[cc] = 10 + fact*40
    data.EmissionValueQuad[cc] = 1 + fact*2
    data.InvestValue[cc, "solar"] =25 + 50*fact 
    data.InvestValue[cc, "wind"] = 5 + 20*fact
    return data

def emmInvc1(data, scen):
    return emmInv(data, scen, "c1")

def emmInvc2(data, scen):
    return emmInv(data, scen, "c2")

def crossCoeff(data, scen, cc):
    if scen == -1:
        data.EmissionCrossValue[cc] = 0
        for ee in data.green:
            data.InvestCrossValue[cc,ee] = 0
        return data
    data.EmissionCrossValue[cc] = data.EmissionValueQuad[cc]*(1+scen)
    data.InvestCrossValue[cc,"wind"] = data.InvestValue[cc,"wind"]*(1+scen)/200
    data.InvestCrossValue[cc,"solar"] = data.InvestValue[cc,"solar"]*(1+scen)/40
    
def crossCoeffc1(data, scen):
    return crossCoeff(data, scen, "c1")

def crossCoeffc2(data, scen):
    return crossCoeff(data, scen, "c2")